In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import pickle

In [2]:
# config of inputs
out_dir = 'data'
out_version = 'v0'

In [3]:
df_collection = pickle.load(open( "{}/Xy_collection_{}.pickle".format(out_dir, out_version), "rb" ))

In [4]:
df_collection.keys()

dict_keys(['X_valid_ica_full_MinMaxScaler', 'X_valid_pca_full_StandardScaler', 'X_train_raw_MinMaxScaler', 'X_test_raw_StandardScaler', 'X_valid_pca_full_MinMaxScaler', 'X_test_pca_full_MinMaxScaler', 'X_test_ica_full_MinMaxScaler', 'y_train', 'X_valid_pca_mini_MinMaxScaler', 'X_test_raw_MinMaxScaler', 'X_valid_raw', 'X_valid_raw_StandardScaler', 'X_test_pca_full_StandardScaler', 'X_train_ica_mini_StandardScaler', 'X_test_pca_mini', 'X_valid_raw_MinMaxScaler', 'X_valid_pca_trunk', 'X_train_pca_trunk_MinMaxScaler', 'X_train_ica_full', 'X_valid_pca_trunk_StandardScaler', 'X_test_ica_mini_StandardScaler', 'X_train_pca_full_StandardScaler', 'X_test_ica_mini', 'X_test_ica_mini_MinMaxScaler', 'X_train_pca_trunk_StandardScaler', 'X_train_ica_mini_MinMaxScaler', 'X_test_pca_trunk_MinMaxScaler', 'X_valid_pca_full', 'X_test_pca_trunk_StandardScaler', 'X_train_pca_mini', 'X_train_pca_mini_MinMaxScaler', 'X_valid_pca_mini_StandardScaler', 'X_test_pca_mini_MinMaxScaler', 'X_test_pca_mini_StandardSc

In [5]:
transforms = [s[len('X_train_'):] for s in  df_collection.keys() if 'X_train' in s]

In [6]:
transforms

['raw_MinMaxScaler',
 'ica_mini_StandardScaler',
 'pca_trunk_MinMaxScaler',
 'ica_full',
 'pca_full_StandardScaler',
 'pca_trunk_StandardScaler',
 'ica_mini_MinMaxScaler',
 'pca_mini',
 'pca_mini_MinMaxScaler',
 'pca_mini_StandardScaler',
 'ica_full_StandardScaler',
 'raw',
 'ica_mini',
 'pca_trunk',
 'ica_full_MinMaxScaler',
 'pca_full_MinMaxScaler',
 'pca_full',
 'raw_StandardScaler']

In [7]:
!python --version

Python 3.5.5


# Vowpal Wabbit

In [8]:
# Added to suppress warnings from model fiting of type
# DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. 
import warnings
warnings.filterwarnings('ignore', 'The truth value of an empty array is ambiguous. .*')

In [9]:
from vowpalwabbit.sklearn_vw import VWClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

from sklearn.metrics import make_scorer

from sklearn.metrics import accuracy_score

In [13]:
clf = VWClassifier(quiet=False)
clf.fit(df_collection['X_train_{}'.format('raw')], df_collection['y_train'])

{'loss_function': 'logistic', 'quiet': False}

In [14]:
clf.predict(df_collection['X_valid_{}'.format('raw')])

array([-1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,
       -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,
       -1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1.,
        1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,
       -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
        1.,  1., -1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1.,
        1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1.,
       -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,
        1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,
       -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
       -1., -1., -1.,  1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1.,
       -1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1., -1.,  1

In [ ]:
cv_results=[]
xgb_param_test_list = [{'max_depth': [2,3,4,5,6,8,10],
                    'min_child_weight': sp_randint(1, 10),
                    'subsample': sp_uniform(loc=0.6, scale=0.4), 
                    'colsample_bytree': sp_uniform(loc=0.85, scale=0.15),
                    'reg_alpha': [0, 1e-5, 1e-3, 1e-1, 1, 3, 5, 10],
                    'reg_lambda': [0, 1e-5, 1e-3, 1e-1, 1, 5, 10]}]

n_iter = [1000]
clf=xgb.XGBClassifier(max_depth=-1, random_state=314, seed=314, silent=True, metric='None', n_jobs=2)

for n, param_test in zip(n_iter, xgb_param_test_list):
    print('============================ ', n, ' =================')
    gs1 = RandomizedSearchCV(estimator=clf, param_distributions=param_test, 
                             n_iter=n,
                             scoring=make_scorer(accuracy_score, greater_is_better=True),
                             cv=4,
                             random_state=314,
                             verbose=True)
    #gs1.fit(df_collection['X_train_{}'.format('raw')] , df_collection['y_train'])
    gs1.fit(pd.concat([df_collection['X_train_{}'.format('raw')],
                       df_collection['X_valid_{}'.format('raw')]]), 
            pd.concat([df_collection['y_train'],
                       df_collection['y_valid']]))
    print(gs1.best_params_)
    print(gs1.best_score_)
    cv_results.append(gs1.cv_results_)

In [ ]:
for i in np.argsort(cv_results[0]['mean_test_score'])[-5:]:
    print('{1} {2}   :  {0}'.format(cv_results[0]['params'][i], 
                                    cv_results[0]['mean_test_score'][i], 
                                    cv_results[0]['mean_train_score'][i]))

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
score = cross_validate(xgb.XGBClassifier(max_depth=3, random_state=314, silent=True, metric='None', n_jobs=2),
                       X=df_collection['X_train_{}'.format('raw')],
                       y=df_collection['y_train'],
                       scoring=make_scorer(accuracy_score, greater_is_better=True),
                       cv=4,
                       return_train_score=True)

In [ ]:
score

# LightGBM

In [9]:
from lightgbm.sklearn import LGBMClassifier

In [10]:
lgb_cv_results = [] 
lgb_param_test_list = [{'max_depth': [-1],
                    'num_leaves': sp_randint(4, 10),
                    'min_child_weight': sp_randint(1, 10),
                    'colsample_bytree': sp_uniform(loc=0.6, scale=0.4), 
                    'subsample': sp_uniform(loc=0.6, scale=0.4),
                    'reg_alpha': [0, 1e-5, 1e-3, 1e-1, 1],
                    'reg_lambda': [0, 1e-5, 1e-3, 1e-1, 1, 2]}]

n_iter = [1000]
clf=lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=2)

for n, param_test in zip(n_iter, lgb_param_test_list):
    print('============================ ', n, ' =================')
    gs2 = RandomizedSearchCV(estimator=clf, param_distributions=param_test, 
                             n_iter=n,
                             scoring=make_scorer(accuracy_score, greater_is_better=True),
                             cv=4,
                             random_state=314,
                             verbose=True)
    gs2.fit(df_collection['X_train_{}'.format('raw')] , df_collection['y_train'])
    print(gs2.best_params_)
    print(gs2.best_score_)
    lgb_cv_results.append(gs2.cv_results_)

============================  1000  =================
Fitting 4 folds for each of 1000 candidates, totalling 4000 fits


/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

...

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151

{'colsample_bytree': 0.7586985504855354, 'max_depth': -1, 'min_child_weight': 6, 'num_leaves': 8, 'reg_alpha': 0.1, 'reg_lambda': 0, 'subsample': 0.7901225786566077}
0.8341384863123994


[Parallel(n_jobs=1)]: Done 4000 out of 4000 | elapsed:  2.0min finished


In [11]:
for i in np.argsort(lgb_cv_results[0]['mean_test_score'])[-5:]:
    print('{1} {2}   :  {0}'.format(lgb_cv_results[0]['params'][i], 
                                    lgb_cv_results[0]['mean_test_score'][i], 
                                    lgb_cv_results[0]['mean_train_score'][i]))

0.8293075684380032 0.8856762707715647   :  {'colsample_bytree': 0.6397218438687312, 'max_depth': -1, 'min_child_weight': 4, 'num_leaves': 8, 'reg_alpha': 1e-05, 'reg_lambda': 0.1, 'subsample': 0.6854007145063687}
0.8309178743961353 0.9087553732065554   :  {'colsample_bytree': 0.9371065872197792, 'max_depth': -1, 'min_child_weight': 1, 'num_leaves': 8, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.8224905294109072}
0.8325281803542673 0.9130460798011903   :  {'colsample_bytree': 0.8868633336997939, 'max_depth': -1, 'min_child_weight': 1, 'num_leaves': 9, 'reg_alpha': 0.001, 'reg_lambda': 1e-05, 'subsample': 0.823515429018327}
0.8325281803542673 0.9055318791397493   :  {'colsample_bytree': 0.7706609849751354, 'max_depth': -1, 'min_child_weight': 1, 'num_leaves': 8, 'reg_alpha': 1e-05, 'reg_lambda': 1e-05, 'subsample': 0.8462952920530427}
0.8341384863123994 0.8786996896957465   :  {'colsample_bytree': 0.7586985504855354, 'max_depth': -1, 'min_child_weight': 6, 'num_leaves': 8, 'reg_a

/home/mlisovyi/anaconda2/envs/iml2018/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(acc_train_summary, cmap='Blues', annot=True, vmin=0.75, vmax=0.9)
plt.title('Train sample')